In [1]:
import os
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/bil96/Wine-Quality.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "bil96"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "99cc54f8815271eb745f16c20e6b33fe22603ab9"

In [2]:
%pwd

'c:\\Users\\bilal\\Documents\\mlops\\Wine-Quality\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\bilal\\Documents\\mlops\\Wine-Quality'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params_path: Path
    metrics_file_name: str
    target_column: str
    mlflow_uri: str


In [6]:
from src.WineQuality.constants import *
from src.WineQuality.utils.common import read_yaml, create_directories ,save_json


In [7]:
%pwd

'c:\\Users\\bilal\\Documents\\mlops\\Wine-Quality'

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH,
                 ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])


        model_eval_config = ModelEvaluationConfig(
            root_dir=self.config.artifacts_root,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params_path=params,
            metrics_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/bil96/Wine-Quality.mlflow"
        )
        return model_eval_config

In [9]:
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [12]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate_metrics(self,actual, predicted ):
        r2 = r2_score(actual, predicted)
        mae = mean_absolute_error(actual, predicted)
        mse = mean_squared_error(actual, predicted)
        rmse = np.sqrt(mse)

        return r2, mae, mse, rmse

    def log_into_mlflow(self ):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        X_test = test_data.drop(columns=[self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predict = model.predict(X_test)
            (r2, mae, mse, rmse) = self.evaluate_metrics(y_test, predict)
            scores = {"r2_score": r2, "mean_absolute_error": mae, "mean_squared_error": mse, "root_mean_squared_error": rmse}
            save_json(path=Path( self.config.metrics_file_name), data=scores)
            mlflow.log_params(self.config.all_params_path)

            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("r2",r2)
            mlflow.log_metric("mae",mae)

            if tracking_uri == "file":
                mlflow.sklearn.log_model(model,"model" ,registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")





In [13]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    model_eval_config = ModelEvaluation(config=model_eval_config)
    model_eval_config.log_into_mlflow()
except Exception as e:
    raise e


[2025-05-13 03:50:00,997: INFO:wine-quality:27: YAML file config\config.yaml loaded successfully.]
[2025-05-13 03:50:00,999: INFO:wine-quality:27: YAML file params.yaml loaded successfully.]
[2025-05-13 03:50:01,004: INFO:wine-quality:27: YAML file schema.yaml loaded successfully.]
[2025-05-13 03:50:01,006: INFO:wine-quality:48: Creating directory: artifacts]
[2025-05-13 03:50:01,009: INFO:wine-quality:48: Creating directory: artifacts/model_evaluation]
[2025-05-13 03:50:01,211: INFO:wine-quality:61: JSON file saved at artifacts\model_evaluation\metrics.json]


2025/05/13 03:50:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run rogue-duck-940 at: https://dagshub.com/bil96/Wine-Quality.mlflow/#/experiments/0/runs/3b1081f02df64e4db7ebbb5609ada6d8
🧪 View experiment at: https://dagshub.com/bil96/Wine-Quality.mlflow/#/experiments/0
